### 1. Introduction

This notebook presents a streamlined document intelligence and Retrieval-Augmented Generation (RAG) pipeline tailored for German banking regulations. It ingests authentic regulatory PDFs and transforms them into a searchable, structured knowledge base.

### 2. Why This Approach

The Free Edition does not support DBFS, Vector Search, or managed PDF extraction.  
To address these limitations, we implemented a custom pipeline leveraging:

- UC Volume storage
- Binary file reading
- Token-based chunking (mapInPandas)
- FAISS-powered vector search
- OpenAI embeddings
- GPT-4o-mini for answer synthesis

This architecture is fully operational on the Free Edition and remains compatible with Databricks’ upcoming built-in PDF intelligence stack.

### 3. Pipeline Overview

##### Ingestion → Extraction → Chunking → Embedding → FAISS Search → RAG Answer

Each stage features robust safeguards, including:

- Token-length validation
- Chunk-length inspection
- Similarity ranking
- Transparent referencing of sources

### 4. Validation Notes

- Extracted text snippets were cross-checked with page headers to ensure accuracy.
- Chunk sizes were validated (maximum ~3,200 characters).
- Embeddings were checked to prevent input overflow.
- Similarity ranking reliably retrieves semantically relevant sections (e.g., ESG queries return ESG content).

### 5. Future State

In a full Databricks deployment, this notebook can be seamlessly upgraded to leverage:

- **ai_parse_document** for advanced document parsing
- **Databricks Vector Search** for scalable semantic retrieval
- **Agent Framework & Knowledge Assistants** for interactive, context-aware querying
- **Lakehouse Monitoring** to ensure data and answer quality
- **Workflow Jobs** for end-to-end automation

This PoC is intentionally minimal, yet fully compatible with future enhancements.

In [0]:
# The following code installs required Python packages for PDF processing, tokenization, embeddings, and OpenAI API access.
# It also imports necessary libraries for handling files, dataframes, and Spark DataFrame schema definitions.

#%pip install pypdf PyPDF2 tiktoken faiss-cpu openai  # Installs libraries for PDF reading, tokenization, vector search, and OpenAI API

import os  # For operating system interactions
import pandas as pd  # For data manipulation with Pandas DataFrames
import numpy as np  # For numerical operations, especially with embeddings
from io import BytesIO  # For handling byte streams, useful for reading PDF files from binary content
from pypdf import PdfReader  # For extracting text from PDF files

import tiktoken  # For tokenizing text, especially for OpenAI models
from pyspark.sql.functions import col  # For Spark DataFrame column operations
from pyspark.sql.types import StructType, StructField, StringType  # For defining Spark DataFrame schemas

In [0]:
# Create a Databricks widget to input the OpenAI API key.
# This allows users to securely provide their API key for authentication,
# which is needed to access OpenAI services in later cells.
dbutils.widgets.text("openai_key", "", "Enter OpenAI API Key")

In [0]:
# Define paths for PDF documents and extracted text storage.
# VOLUME_PATH: Base directory for workspace data.
# PDF_PATH: Location of PDF files to process.
# EXTRACTED_DELTA_PATH: Destination for extracted text in Delta format.
VOLUME_PATH = "/Volumes/workspace/hackathoncompliancedoc"
PDF_PATH = VOLUME_PATH + "/documents"
EXTRACTED_DELTA_PATH = VOLUME_PATH + "/extracted_delta"

print("PDFs path:", PDF_PATH)
print("Delta path for extracted text:", EXTRACTED_DELTA_PATH)

PDFs path: /Volumes/workspace/hackathoncompliancedoc/documents
Delta path for extracted text: /Volumes/workspace/hackathoncompliancedoc/extracted_delta


In [0]:
# Read PDF files as binary from the specified directory into a Spark DataFrame.
# This allows us to process the raw content of each PDF for further extraction and analysis.
pdf_df = spark.read.format("binaryFile").load(PDF_PATH)
display(pdf_df.limit(5))

path,modificationTime,length,content


In [0]:
# This code extracts text from PDF files stored as binary in a Spark DataFrame.
# It uses a UDF to convert PDF binary content to text, then saves the results as a Delta table.

from io import BytesIO
from PyPDF2 import PdfReader
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

def extract_text_from_bytes(byte_array):
    pdf_stream = BytesIO(byte_array)
    reader = PdfReader(pdf_stream)
    text = ""
    for page in reader.pages:
        text += (page.extract_text() or "") + "\n"
    return text

extract_udf = udf(extract_text_from_bytes, StringType())

extracted_df = pdf_df.withColumn(
    "text",
    extract_udf(col("content"))
).select(
    col("path"),
    col("text")
)

# Save extracted text to Delta format for efficient downstream processing.
extracted_df.write.format("delta").mode("overwrite").save(EXTRACTED_DELTA_PATH)

display(extracted_df.limit(5))

path text dbfs:/Volumes/workspace/hackathoncompliancedoc/documents/Final Guidelines on the management of ESG risks.pdf 
 
 
EBA/ GL/2025/01 
08/01/2025 
 
 
Final Report 
Guidelines o n the management of environmental, social and 
governance (ESG) risks 
 
 
 
 
 
 
FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 
 
2 
 
Contents 
1. Executive Summary 3 
2. Background and rationale 4 
3. Guidelines 13 
4. Reference methodology for the identification and measurement of ESG risks 18 
5. Minimum standards and reference methodology for the management and monitoring of 
ESG risks 27 
6. Plans in accordance with Article 76(2) of Directive 2013/36 /EU 38 
7. Accompanying documents 46 
 
 
FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 
 
3 
 
 
1. Executive Summary 
The EBA is mandated in accordance with Article 87a(5) of Directive 2013/36/EU to issue guidelines 
on minimum standards and reference methodologies for the identification, measurement, 
management and monitoring of environmental, social and governance (ESG) risks by institutions . 
ESG risks , in particular environmental risks through transition and physical risk drivers , pos e 
challenges to the safety and soundness of institutions and may affect all traditional categories of 
financial risks to which they are exposed. To ensure the resilience of the business model and risk 
profile of institutions in the short, medium and long term, the guidelines set requirements for the 
internal processes and ESG risk management arrangements that institutions should have in place . 
Institutions, based on regular and comprehensive materiality assessments of ESG risks , should 
ensure that they are able to properly identify and measure ESG risks through sound data processes 
and a combination of methodologies, including exposure -, portfolio - and sector -based , portfolio 
alignment and scenario -based methodologies. 
Institutio ns should integrate ESG risks in to their regular risk management framework by considering 
their role as potential drivers of all traditional categories of financial risks, including credit, market, 
operational, reputational, liquidity, business model, and concentration risks. Institutions should 
have a robust and sound approach to managing and mitigating ESG risks over the short, medium 
and long term , including a time horizon of at least 10 years , and should apply a range of risk 
management tools including engagement with counterparties . Institutions should embed ESG risks 
in th eir regular processes including in the risk appetite, internal controls and ICAAP. Besides, 
institutions should monitor ESG risks through effective internal reporting frameworks and a range 
of backward - and forward -looking ESG risk metrics and indicators. 
Institutions should develop specific plans to address the risks arising from the transition and process 
of adjustment of the economy towards the regulatory objectives related to ESG factors of the 
jurisdictions they operate in . To this end, institutions should assess and embed forward -looking ESG 
risk considerations in their strategies , policies and risk management processes through transition 
planning considering short -, medium - and long -term time horizons. CRD -based plans take a risk -
based view and contribute to the overall resilience of institutions towards ESG risk s and should be 
consistent with transition plans prepared or d isclosed by institutions under other pieces of EU 
legislation . 
Next steps 
The guidelines will apply from 11 January 2026 except for small and non -complex institutions for 
which the guidelines will apply at the lates t from 11 January 2027. 
FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 
 
4 
 
2. Background and rationale 
2.1 Impact of ESG risks 
1. Climate change, environmental degradation, biodiversity loss, social issues and other 
environmental, social and governance (ESG) factors pos e considerable challenges for the 
economy. The imp

In [0]:
# Initialize the tokenizer for OpenAI models using the "cl100k_base" encoding.
# This tokenizer is used to split text into tokens compatible with OpenAI's embedding models.
tokenizer = tiktoken.get_encoding("cl100k_base")

# Set the maximum number of tokens per chunk for embedding.
# OpenAI's embedding models have a token limit; using 500 tokens ensures each chunk is safely within the model's constraints.
MAX_TOKENS = 500  # safe for embedding

In [0]:
# This function splits extracted PDF text into smaller chunks suitable for embedding and downstream processing.
# It iterates over batches of rows, tokenizes the text, and slices it into segments of MAX_TOKENS length.
# Each chunk is associated with its document path for traceability.
# The output is a DataFrame with columns: doc_path and chunk_text.
# This chunking is necessary because embedding models have a maximum token limit per input,
# and smaller chunks improve retrieval accuracy and efficiency in RAG (Retrieval-Augmented Generation) workflows.

def chunk_pdf_text(batch_iter):
    import pandas as pd
    rows_out = []

    for batch in batch_iter:
        for _, row in batch.iterrows():
            text = row["text"]
            path = row["path"]

            tokens = tokenizer.encode(text)

            for i in range(0, len(tokens), MAX_TOKENS):
                slice_tokens = tokens[i:i+MAX_TOKENS]
                chunk_text = tokenizer.decode(slice_tokens)
                rows_out.append({
                    "doc_path": path,
                    "chunk_text": chunk_text
                })

    yield pd.DataFrame(rows_out)

schema = StructType([
    StructField("doc_path", StringType(), True),
    StructField("chunk_text", StringType(), True)
])

chunk_final_df = extracted_df.mapInPandas(chunk_pdf_text, schema=schema)
display(chunk_final_df.limit(5))

doc_path,chunk_text
dbfs:/Volumes/workspace/hackathoncompliancedoc/documents/Final Guidelines on the management of ESG risks.pdf,"EBA/ GL/2025/01 08/01/2025 Final Report Guidelines o n the management of environmental, social and governance (ESG) risks FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 2 Contents 1. Executive Summary 3 2. Background and rationale 4 3. Guidelines 13 4. Reference methodology for the identification and measurement of ESG risks 18 5. Minimum standards and reference methodology for the management and monitoring of ESG risks 27 6. Plans in accordance with Article 76(2) of Directive 2013/36 /EU 38 7. Accompanying documents 46 FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 3 1. Executive Summary The EBA is mandated in accordance with Article 87a(5) of Directive 2013/36/EU to issue guidelines on minimum standards and reference methodologies for the identification, measurement, management and monitoring of environmental, social and governance (ESG) risks by institutions . ESG risks , in particular environmental risks through transition and physical risk drivers , pos e challenges to the safety and soundness of institutions and may affect all traditional categories of financial risks to which they are exposed. To ensure the resilience of the business model and risk profile of institutions in the short, medium and long term, the guidelines set requirements for the internal processes and ESG risk management arrangements that institutions should have in place . Institutions, based on regular and comprehensive materiality assessments of ESG risks , should ensure that they are able to properly identify and measure ESG risks through sound data processes and a combination of methodologies, including exposure -, portfolio - and sector -based , portfolio alignment and scenario -based methodologies. Institutio ns should integrate ESG risks in to their regular risk management framework by considering their role as potential drivers of all traditional categories of financial risks, including credit, market, operational, reputational, liquidity, business model, and concentration risks. Institutions should have a robust and sound approach to managing and mitigating ESG risks over the short, medium and long term"
dbfs:/Volumes/workspace/hackathoncompliancedoc/documents/Final Guidelines on the management of ESG risks.pdf,", including a time horizon of at least 10 years , and should apply a range of risk management tools including engagement with counterparties . Institutions should embed ESG risks in th eir regular processes including in the risk appetite, internal controls and ICAAP. Besides, institutions should monitor ESG risks through effective internal reporting frameworks and a range of backward - and forward -looking ESG risk metrics and indicators. Institutions should develop specific plans to address the risks arising from the transition and process of adjustment of the economy towards the regulatory objectives related to ESG factors of the jurisdictions they operate in . To this end, institutions should assess and embed forward -looking ESG risk considerations in their strategies , policies and risk management processes through transition planning considering short -, medium - and long -term time horizons. CRD -based plans take a risk - based view and contribute to the overall resilience of institutions towards ESG risk s and should be consistent with transition plans prepared or d isclosed by institutions under other pieces of EU legislation . Next steps The guidelines will apply from 11 January 2026 except for small and non -complex institutions for which the guidelines will apply at the lates t from 11 January 2027. FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 4 2. Background and rationale 2.1 Impact of ESG risks 1. Climate change, environmental degradation, biodiversity loss, social issues and other environmental, social and governance (ESG) factors pos e considerable challeng

In [0]:
# This code displays the 20 longest text chunks from the chunk_final_df DataFrame.
# It calculates the length of each chunk, extracts a 150-character sample, and orders by length descending.
# This helps inspect chunk sizes and verify chunking logic for embedding and retrieval.

display(
    chunk_final_df.selectExpr(
        "length(chunk_text) AS len",
        "substr(chunk_text,1,150) AS sample"
    ).orderBy(col("len").desc()).limit(20)
)

len,sample
3371,Federal Financial Supervisory Authority (BaFin) Minimum Requirements for Risk Management (MaRisk) – Page 1 of 22 BaFin Translation - The present Eng
2981,14 BTO 1.2.5 Treatment of problem loans ..............................................14 BTO 1.2.6 Risk provisioning .............................
2623,"the general organisational requirements pursuant to Art. 5, as well as the risk management and internal audit requirements pursuant to Art. 7 and 8,"
2620,.2 Internal processes and capacity 91. Institutions should ensure meaningful and regular interaction and exchanges at all levels of the organi sati
2606,BA has identified largely divergent practices of banks when handling forward - looking information for risk assessments . Robust materiality assessme
2588,"certain sectors as materially exposed to ESG risks but to emphasise banks’ responsibility to conduct robust assessments. Therefore, the preferred o"
2585,", by having regard to applicable local legislation and ESG regulatory objectives, and should be able to demonstrate a wel l- informed consolidated ap"
2559,"the calibration of their liquidity buffers or their liquidity risk management framework. 72. In addition, with regard to environmental risks, ins"
2555,/legal -content/EN/TXT/?uri=CELEX:32023H1425 FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 8 requirements for the monitoring and
2552,", however, include a horizon that is long enough to cover for those risks that may fully materiali se in the long term. The preferred option is Optio"


In [0]:
# Convert the Spark DataFrame 'chunk_final_df' to a Pandas DataFrame for local processing and inspection.
# This is useful for tasks that require Pandas functionality, such as embedding generation, similarity search, or visualization.
chunks_pd = chunk_final_df.toPandas()
chunks_pd.head()  # Display the first few rows to verify the conversion and inspect the chunked text data.

,doc_path,chunk_text
0,dbfs:/Volumes/workspace/hackathoncompliancedoc...,\n \n \nEBA/ GL/2025/01 \n08/01/2025 \n \n \...
1,dbfs:/Volumes/workspace/hackathoncompliancedoc...,", including a time horizon of at least 10 yea..."
2,dbfs:/Volumes/workspace/hackathoncompliancedoc...,"economy , as enshrined in the Paris Agreement,..."
3,dbfs:/Volumes/workspace/hackathoncompliancedoc...,CRD and in the CRR. In particular and to ensu...
4,dbfs:/Volumes/workspace/hackathoncompliancedoc...,"the short, medium and long -term, to be prepar..."


In [0]:
# This function generates a vector embedding for a given text using OpenAI's embedding model.
# It first tokenizes the input text and checks if the number of tokens exceeds MAX_TOKENS.
# If the text is too long, it raises an error to prevent issues with the embedding API.
# Then, it calls OpenAI's API to get the embedding and returns it as a NumPy array.
# Embeddings are used for semantic search, similarity comparison, and retrieval in RAG workflows.
import openai
openai.api_key = dbutils.widgets.get("openai_key")

def get_embedding_safe(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return np.array(response.data[0].embedding, dtype="float32")


In [0]:
# This code builds a FAISS index for fast similarity search over document chunk embeddings.
# It stacks all chunk embeddings into a matrix, initializes a FAISS index for L2 distance,
# and adds the embeddings to the index. This enables efficient retrieval of the most similar
# document chunks to a query embedding, which is essential for RAG workflows.
# Generate embeddings for each chunk
chunks_pd["embedding"] = chunks_pd["chunk_text"].apply(get_embedding_safe)

print("Generated embeddings:", len(chunks_pd))
print("Sample embedding size:", len(chunks_pd['embedding'].iloc[0]))

import faiss

embedding_matrix = np.stack(chunks_pd["embedding"].values)
dimension = embedding_matrix.shape[1]

faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embedding_matrix)

print("FAISS index size:", faiss_index.ntotal)

Generated embeddings: 413
Sample embedding size: 1536
FAISS index size: 413


In [0]:
# This function retrieves the top-k most similar document chunks to a given query using FAISS.
# Steps:
# 1. Generate an embedding for the query using OpenAI's embedding model.
# 2. Search the FAISS index for the k nearest document chunk embeddings.
# 3. For each result, collect its rank, similarity score, source document path, and a text snippet.
# 4. Return the results as a Pandas DataFrame for inspection or downstream use.
# This is used to efficiently find the most relevant document chunks for a query, supporting semantic search and RAG workflows.

def retrieve_top_k(query, k=5):
    q_embed = get_embedding_safe(query).reshape(1, -1)
    D, I = faiss_index.search(q_embed, k)

    out = []
    for rank, idx in enumerate(I[0]):
        out.append({
            "rank": rank + 1,
            "score": float(D[0][rank]),
            "source": chunks_pd.iloc[idx]["doc_path"],
            "text_snippet": chunks_pd.iloc[idx]["chunk_text"][:500]
        })
    return pd.DataFrame(out)

# Test
retrieve_top_k("What are BaFin ESG risk management requirements?")

,rank,score,source,text_snippet
0,1,0.755549,dbfs:/Volumes/workspace/hackathoncompliancedoc...,"the short, medium and long -term, to be prepar..."
1,2,0.762324,dbfs:/Volumes/workspace/hackathoncompliancedoc...,"economy , as enshrined in the Paris Agreement,..."
2,3,0.769957,dbfs:/Volumes/workspace/hackathoncompliancedoc...,BA considers that there is a need to incorpora...
3,4,0.775456,dbfs:/Volumes/workspace/hackathoncompliancedoc...,climate -related financial risks10. They also...
4,5,0.778710,dbfs:/Volumes/workspace/hackathoncompliancedoc...,it is important that institutions progressive...


In [0]:
# This function implements a Retrieval-Augmented Generation (RAG) workflow for compliance Q&A.
# It retrieves the most relevant document chunks for a query, builds a context, and prompts an LLM for an answer.
# Steps:
# 1. Copy chunked document data for local processing.
# 2. Generate an embedding for the query using OpenAI's embedding model.
# 3. Compute cosine similarity between the query embedding and each chunk embedding.
# 4. Select the top 3 most similar chunks as context.
# 5. Construct a prompt with the context and query, instructing the LLM to answer using only the provided context.
# 6. Call OpenAI's chat completion API to generate a concise, compliance-focused answer.
# 7. Return the answer and the source chunks used for context.
# This approach ensures answers are grounded in source documents, supporting trustworthy and explainable compliance responses.

def generate_rag_answer(query):
    local_chunks = chunks_pd.copy()
    query_embedding = get_embedding_safe(query)

    local_chunks["similarity"] = local_chunks["embedding"].apply(
        lambda emb: float(
            np.dot(emb, query_embedding) /
            (np.linalg.norm(emb) * np.linalg.norm(query_embedding))
        )
    )

    top_chunks = local_chunks.sort_values("similarity", ascending=False).head(3)

    context = "\n\n".join(top_chunks["chunk_text"].tolist())

    prompt = f"""
    You are a German banking compliance assistant (BaFin, MaRisk, GwG, CSRD, ESRS).

    Use ONLY the following context to answer the question.

    CONTEXT:
    {context}

    QUESTION:
    {query}

    Provide a short, correct summary referencing BaFin/ESG concepts.
    """

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=250
    )

    answer = response.choices[0].message.content

    source_cols = ["chunk_text", "similarity"]
    if "doc_path" in top_chunks.columns:
        source_cols.append("doc_path")

    sources = top_chunks[source_cols].reset_index(drop=True)

    return answer, sources

In [0]:
# The following code demonstrates how to use a Retrieval-Augmented Generation (RAG) workflow
# to answer a compliance-related query using document chunks and an LLM (Large Language Model).
# 1. The query is defined as a string about BaFin ESG risk management requirements.
# 2. The generate_rag_answer function is called, which retrieves relevant document chunks,
#    builds a context, and prompts the LLM to generate a concise answer.
# 3. The answer is printed, followed by the source chunks used for context, displayed in a rich format.
# This process ensures that the answer is grounded in the source documents and references
# BaFin/ESG compliance concepts, supporting trustworthy and explainable responses.

query = "What are BaFin ESG risk management requirements?"
answer, sources = generate_rag_answer(query)

print("ANSWER:\n")
print(answer)

print("\nSOURCES (all columns):")
display(sources)

ANSWER:

BaFin's ESG risk management requirements emphasize that institutions must systematically identify, measure, and manage environmental, social, and governance (ESG) risks. This is essential due to the forward-looking nature of ESG risks and their potential impacts over various time horizons. Institutions are expected to integrate ESG considerations into their business models and strategies, ensuring they account for short, medium, and long-term challenges. 

Key aspects include:

1. Development of robust metrics and risk indicators for ESG risks, necessitating strong data processes.
2. Consideration of different time horizons when defining business models.
3. The necessity for institutions to utilize both available data and improve the collection of ESG-related information from clients and counterparties.
4. Compliance with regulatory frameworks, including adherence to guidelines issued by the European Banking Authority (EBA) concerning minimum standards for the management of ES

chunk_text,similarity,doc_path
"the short, medium and long -term, to be prepared in accordance with Article 76(2). This assessment shall take into account the institutions’ sustainability -related product offering, their transition finance policies, related loan origination policies, and ESG-related tar gets and limits. Competent authorities shall assess the robustness of those plans as part of the supervisory review and evaluation process. FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 6 Where relevant, for the assessment referred to in the first subparagraph, competent authorities may cooperate with authorities or public bodies in charge of climate change and environmental supervision. 7. To foster robust risk management practices and ensure convergence across the Union , the EBA has been empowered in Article 87a (5) of the CRD to issue guidelines to specify : a) minimum standards and reference methodologies for the identification, measurement, management and monitoring of ESG risks; b) the content of plans to be prepared in accordance with Article 76 (2) of the CRD, which shall include specific timelines and intermediate quantifiable targets and milestones, in order to monitor and address the financial risks stemming from ESG factors , including those arising from the process of adjustment and transition trends towards the relevant Member States and Union regulatory objectives in relation to ESG factors, in particular the objective to achieve climate neutrality by 2050 as set out in Regulation (EU) 2021/1119, as well as, where relevant for internationally active institu tions, third - country legal and regulatory objectives ; c) qualitative and quantitative criteria for the assessment of the impact of ESG risks on the risk profile and solvency of institutions in the short, medium and long term; d) criteria for setting the scenarios referred to in paragraph 3 of Article 87 a of the CRD , including the parameters and assumptions to be used in each of the scenarios , specific risks and time horizons . 8. These guidelines address the aspects included in points a), b) and c ) of the mandate entrusted to the EBA. Point d) of the mandate will be addressed through the development of complementary guidelines on scenario analysis related to ESG factors . Therefore, these guidelines on the management of ESG",0.6222257018089294,dbfs:/Volumes/workspace/hackathoncompliancedoc/documents/Final Guidelines on the management of ESG risks.pdf
"economy , as enshrined in the Paris Agreement, the United Nations 2030 Agenda for Sustainable Development and t he European Green Deal , and for managing the financial risks that this transition may entail and /or those stemming from other ESG factors. 3. Environmental risks , including climate -related risks , are expected inter alia to become even more prominent going forward through different possible combinations of transition and physical risks. Th ese may affect all traditional categories of financial risks to which institutions are exposed. In addition , institutions’ counterparties or invested assets may be subject to the negative impact of social factors , such as breaches of human rights, demographic change , digitalisation, health or working conditions , and governanc e factors , such as shortcomings in executive leadership or bribery and corruption , which may in turn lead to financial risks that institutions should assess and manage . 4. To maintain adequate resilience to the negative impacts of ESG factors, institutions established in the EU need to be able to systematically identify, measure and manage ESG risks. However , the specificities of ESG risks such as their forward -looking nature and distinct impacts over various time horizons , as well as the lack of relevant historical experience , means that understanding, measurement and management practices can differ significantly across institutions. The EBA’s observations stemming from the monitoring of supervisory colleges , 

In [0]:
# The following code demonstrates how to use a Retrieval-Augmented Generation (RAG) workflow
# to answer a compliance-related query using document chunks and an LLM (Large Language Model).
# 1. The query is defined as a string about BaFin ESG risk expectations.
# 2. The generate_rag_answer function is called, which retrieves relevant document chunks,
#    builds a context, and prompts the LLM to generate a concise answer.
# 3. The answer is printed, followed by the source chunks used for context, displayed in a rich format.
# This process ensures that the answer is grounded in the source documents and references
# BaFin/ESG compliance concepts, supporting trustworthy and explainable responses.

query = "What are BaFin ESG risk expectations?"
answer, sources = generate_rag_answer(query)

print(answer)
display(sources)

BaFin's expectations regarding ESG risks are aligned with the guidelines set forth by the EBA in the context of managing financial risks associated with environmental, social, and governance (ESG) factors. Institutions are expected to systematically identify, measure, and manage ESG risks as part of their overall risk management frameworks. This involves assessing the potential financial impacts of ESG factors, including climate-related risks, on their risk profile and solvency over short, medium, and long-term horizons. 

Furthermore, BaFin emphasizes the integration of ESG considerations into institutions' business strategies, loan origination policies, and governance arrangements. Regulatory bodies may collaborate with environmental authorities to ensure robust risk management practices, and institutions must prepare comprehensive plans that include specific targets and timelines for addressing these risks, in line with the EU's objectives for achieving climate neutrality by 2050. O

chunk_text,similarity,doc_path
"the short, medium and long -term, to be prepared in accordance with Article 76(2). This assessment shall take into account the institutions’ sustainability -related product offering, their transition finance policies, related loan origination policies, and ESG-related tar gets and limits. Competent authorities shall assess the robustness of those plans as part of the supervisory review and evaluation process. FINAL REPORT ON GUIDELINES ON THE MANAGEMENT OF ESG RISKS 6 Where relevant, for the assessment referred to in the first subparagraph, competent authorities may cooperate with authorities or public bodies in charge of climate change and environmental supervision. 7. To foster robust risk management practices and ensure convergence across the Union , the EBA has been empowered in Article 87a (5) of the CRD to issue guidelines to specify : a) minimum standards and reference methodologies for the identification, measurement, management and monitoring of ESG risks; b) the content of plans to be prepared in accordance with Article 76 (2) of the CRD, which shall include specific timelines and intermediate quantifiable targets and milestones, in order to monitor and address the financial risks stemming from ESG factors , including those arising from the process of adjustment and transition trends towards the relevant Member States and Union regulatory objectives in relation to ESG factors, in particular the objective to achieve climate neutrality by 2050 as set out in Regulation (EU) 2021/1119, as well as, where relevant for internationally active institu tions, third - country legal and regulatory objectives ; c) qualitative and quantitative criteria for the assessment of the impact of ESG risks on the risk profile and solvency of institutions in the short, medium and long term; d) criteria for setting the scenarios referred to in paragraph 3 of Article 87 a of the CRD , including the parameters and assumptions to be used in each of the scenarios , specific risks and time horizons . 8. These guidelines address the aspects included in points a), b) and c ) of the mandate entrusted to the EBA. Point d) of the mandate will be addressed through the development of complementary guidelines on scenario analysis related to ESG factors . Therefore, these guidelines on the management of ESG",0.6001867651939392,dbfs:/Volumes/workspace/hackathoncompliancedoc/documents/Final Guidelines on the management of ESG risks.pdf
"climate -related financial risks10. They also take into account the analysis performed and recommendations included in the EBA Report on the management and supervision of ESG risks , guidance published by supervisors or networks of central banks such as the Network for Greening the Financial System (NGFS) , various initiatives related to transition planning and plans11 as well as supervisory experience regarding institutions’ practices on the management of climate and environmental risks. 33. These guidelines are consistent with and include cross -references to other EBA guidelines or standards which refer to ESG risks, such as the EBA Guidelines on loan origination and monitoring (with respect to integration of ESG risks in credit risk policies ), the EBA Guidelines on internal governance (with respect to integration of ESG risks in governance arrangements ), and the EBA Implementing Technical Standards on Pillar 3 disclosure of ESG risks (with respect to ESG risk metrics ). In addition, based on t he recent amendments to the CRD, the EBA will introduce or incorporate further ESG risk considerations when developing future guidelines on scenario analysis and when updating its guidelines on internal governance , guidelines on fit-and-proper assessments and guidelines on remuneration policies . These future developments and updates will be done in a way that ensur es consistency with these guidelines on the management of ESG risks , complementing them in specific areas such as scenario analysis,